In [ ]:
RandomForestRegressor()

In [ ]:
def create_model(filters, kernel_size, units, units_gru, dropout_rate, optimizer_model, lstm_act, gru_act, conv_act):
    input_shape = (dataset.get('X_train').shape[1], 1)
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation=conv_act, input_shape=input_shape))
    model.add(LSTM(units, activation=lstm_act, input_shape=input_shape, return_sequences=True))
    model.add(GRU(units_gru, input_shape=input_shape, activation=gru_act))
    model.add(Dropout(rate=dropout_rate))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer=optimizer_model, loss='mean_squared_error')
    return model

best_params = []
mse = []

for dataset in datasets:
    regressor = KerasRegressor(build_fn=create_model, filters=32, kernel_size=3, units=64, units_gru=64, dropout_rate=0.2, optimizer_model='adam', lstm_act='tanh', conv_act='relu', gru_act='relu', epochs=3, batch_size=32)
    grid = GridSearchCV(estimator=regressor, param_grid=param_grid_gru_lstm, n_jobs=7)
    grid_result = grid.fit(dataset.get('X_train'), dataset.get('Y_train'), validation_data = (dataset.get('X_test'), dataset.get('Y_test')))

    best_params.append(grid_result.best_params_)
    print("Лучшее значение параметров: ", grid_result.best_params_)

    score = grid.score(dataset.get('X_test'), dataset.get('Y_test'))
    mse.append(score)
    print("MSE:", score)

print(best_params)
print(mse)
#TODO ген алгоритм на лучших, плюс поправить исходники в обоих ветках